In [139]:
from dotenv import dotenv_values
from langchain_community.llms.vllm import VLLMOpenAI

env_values = dotenv_values("../.env")

llm = VLLMOpenAI(
    openai_api_key=env_values["MISTRAL_API_KEY"],
    openai_api_base=env_values["MISTRAL_HOST"] + "/v1",
    model_name="solidrust/dolphin-2.8-mistral-7b-v02-AWQ",
    model_kwargs={"stop": ["[INST]", "#", "</s>", "[/OUT]", "[INST]", "]"]},
    max_tokens=4096,
    temperature=0.9,
    top_p=1,
)

In [141]:
prompt = """
<s>
[INST] Ты умный помощник, разбирающийся в NER и Entity-level sentiment analysis в финансовых документах. Твоя задача проанализировать предоставленный текст новости, выявить в нём все компании и дать каждой компании оценку в диапазоне от 1 до 5, основываясь на предоставленном тексте, и выдать ответ в формате массива JSON объектов. Отвечай всегда в этом формате, не дописывай ничего лишнего. Пиши всегда на русском.

Чтобы выбрать правильную оценку, следуй следующим правилам:
1: что-то очень негативное относительно компании или дана рекомендация "продавать";
2: что-то скорее негативное, например, вышла отчетность ниже ожиданий, проблемы на каком-нибудь заводе, санкции и пр;
3: Нейтральная новость;
4: Что-то положительное, например, вышла отчетность выше ожиданий, успехи на каком-нибудь заводе, новый контракт;
5: Что-то очень положительное или есть рекомендация "покупать" или "входит в подборку наших супер-акций".

Например, для следующего текста:

"Бесплатных долларов больше нет Акции и инвестиции USDRUB +1.9% После февральских скачков, доллар можно было приобрести практически бесплатно, всего за 50-55 рублей. Товары в РФ на тот момент не упали в цене(и даже выросли), а значит курс валюты был занижен искусственно. Сложно сказать где остановится рост, вполне вероятно мы ещё увидим курс ниже текущего уровня. Рост импорта и выход недружественных нерезидентов с рынка, будут продолжать оказывать поддержку доллару Уже появились первые словесные интервенции Рубль перейдет к укреплению вслед за ростом цен на энергоносители — Силуанов IMOEX -0.1% Без роста курса доллара мы не увидели бы рост индекса от сентябрьским минимумов доллар прибавил более 50%, а индекс прибавил более 40% НЛМК -1.5% Пока не понятно насколько эта информация достоверна Телеграм канал BRIEF сообщил, что санкции против Лисина и НЛМК могут быть включены в 11 пакет санкций Евросоюза. После чего вероятно попадание в SDN-лист. Лично я надеюсь, что это обыкновенный вброс OZON -3.3% Акции падают после публикации хорошей отчётности. Ранее котировки неплохо выросли на ожиданиях отчётности, сегодня идёт фиксация на факте публикации  FIVE -0.2% \X5 Group продолжает изучать возможность смены юрисдикции головной компании из Нидерландов. В данный момент компания не видит возможности для переезда Сейчас все адр-гдр в трудном положении, непонятно как они будут решать эту ситуацию Татнефть ап +1.3% Компания продолжает уверенно идти к полной переработке добываемой нефти Татнефть может купить 50% турецкой топливной компании Aytemiz Akaryakit за $160 млн Сегежа +2.2% «Сегежа групп» планирует разместить в Иркутской области завод по производству CLT-панелей Whoosh +5.5% Компания сегодня выступает на конференции съезд «На СИМ», похоже инвесторам понравилось их выступление. Компания анонсировала новую модель самокатов с интерактивной панелью и ответил на опасения запрета самокатов «как в Париже» Лукойл +0.8% Инвест. дома играют в игру «кто даст больше дивиденд», пока консенсус в районе 500 рублей"

ты должен вернуть:
[/INST]
[
    {{
        "company": "Whoosh WUSH",
        "score": 4.0
    }},
    {{
        "company": "X5 групп",
        "score": 3.0
    }},
    {{
        "company": "OZON",
        "score": 4.0
    }},
    {{
        "company": "НЛМК",
        "score": 2.0
    }}
]
</s>
[INST]
Текст новости:

"{message_text}"

Ты должен вернуть:
[/INST]
"""

In [152]:
message_text = "Windfall Tax — налог на сверхприбыль. Какие компании заплатят Минфину.  Многие понимают, что наш бюджет испытывает трудности и его необходимо наполнять всевозможными способами. Минфин использует различные финансовые рычаги: выход на долговой рынок — ОФЗ, продажа золота и юаней из ФНБ, но одним из главных является налоговый.   Как вы могли заметить, повышение НДПИ (налог на добычу полезных ископаемых) для различных отраслей в этом году уже состоялось. Причём некоторые заплатили разовый уже в прошлом и будут платить в этом: Газпром, Алроса. Это не считая того, что повысили процент на акцизы, штрафы, пошлины, тарифы и сборы.   Windfall Tax — совсем другая история.  Законопроект по налогу пока ещё не опубликован, но у заинтересованных лиц он уже есть на руках, его обсуждают различные издания и комментируют известные личности в финансовых кругах.   Налоговой базой для Windfall Tax будет превышение средней арифметической величины прибыли за 2021 год и за 2022 год над прибылью за 2018 и за 2019 годы.   К плательщикам налога отнесены российские компании, а также иностранные, которые ведут деятельность в России. Конечно, есть исключения, не будут платить: ▪️ Нефтяные и нефтеперерабатывающие компании (приходится доплачивать в бюджет поболее) ▪️ Угольные компании (повышение НДПИ) ▪️ Производители СПГ (до 34% увеличился налог на прибыль для экспортёров СПГ) ▪️ МСП (малые и средние предприятия) ▪️ Компаний с чистой доналоговой прибылью за 2021-2022 годы менее 1₽ млрд или равной этой сумме  Разовый сбор из сверхприбыли российского бизнеса за 2021–2022 годы составит 10% от суммы превышения прибыли над показателем за 2018–2019 годы. При досрочной уплате (с 1 октября по 30 ноября 2023 года) размер Windfall Tax составит 5%. Законопроект вступит в силу с 1 января 2024 года.  Какие же компании заплатят и сколько?  Как вы помните в 2021 году сверхприбыли были у банковского сектора, металлургического, девелоперов и производителей удобрений.   Поэтому наибольший разовый сбор заплатят Сбербанк (20,4₽ млрд), Норникель (18,3₽ млрд) и НЛМК (15₽ млрд).   Такие данные привёл генеральный директор управляющей компании «Грин Лайт Менеджмент» Денис Сверчков. Также в первую десятку плательщиков войдут: ФосАгро (12,1₽ млрд), Русал (10,4₽ млрд), Северсталь (9,5₽ млрд), ММК (9,2₽ млрд), ВТБ (6,8₽ млрд), Полюс (6,3₽ млрд) и ПИК (5,3₽ млрд).   И это при том что у некоторых компаний 2022 год был убыточным, они отказались от выплаты дивидендов и перешли в стадию экономии. Вопрос в том, что дополнительные средства для уплаты Windfall Tax где-то надо изыскать, а значит пройдёт сокращение на инвестпрограммы или дивидендные выплаты.  P.S. Сбербанк оценивает потенциальную выплату Windfall Tax для себя в сумму около 10₽ млрд, воспользуется опцией заплатить досрочно — Герман Греф  С уважением, Владислав Кофанов  Комментируйте на Смартлабе: https://smart-lab.ru/blog/897501.php"

In [153]:
message = prompt.format(message_text=message_text)

In [154]:
resp = llm.invoke(message)

In [155]:
print(resp)

[
    {
        "company": "Sberbank",
        "score": 2.0
    },
    {
        "company": "Nornikel",
        "score": 2.0
    },
    {
        "company": "NLMK",
        "score": 2.0
    },
    {
        "company": "FosAgro",
        "score": 2.0
    },
    {
        "company": "Rusal",
        "score": 2.0
    },
    {
        "company": "Severstal",
        "score": 2.0
    },
    {
        "company": "ММК",
        "score": 2.0
    },
    {
        "company": "ВТБ",
        "score": 2.0
    },
    {
        "company": "Полюс",
        "score": 2.0
    },
    {
        "company": "ПИК",
        "score": 2.0
    }



In [156]:
import json

json.loads(resp + "]")

[{'company': 'Sberbank', 'score': 2.0},
 {'company': 'Nornikel', 'score': 2.0},
 {'company': 'NLMK', 'score': 2.0},
 {'company': 'FosAgro', 'score': 2.0},
 {'company': 'Rusal', 'score': 2.0},
 {'company': 'Severstal', 'score': 2.0},
 {'company': 'ММК', 'score': 2.0},
 {'company': 'ВТБ', 'score': 2.0},
 {'company': 'Полюс', 'score': 2.0},
 {'company': 'ПИК', 'score': 2.0}]